In [44]:
import sys
import os
import subprocess

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext ,Row
from pyspark.sql import functions as F
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
from pyspark.sql.functions import sequence, col, expr, rand, when, lit, concat

from datetime import datetime
import pandas as pd
import random
from datetime import timedelta
import random

In [2]:
spark = SparkSession.builder \
                .appName("spark_session") \
                .config("spark.sql.catalogImplementation", "hive") \
                .config("spark.sql.caseSensitive", "false") \
                .config("hive.exec.dynamic.partition", "true") \
                .config("hive.exec.dynamic.partition.mode", "nonstrict") \
                .config("spark.sql.htl.check", "false") \
                .config("spark.sql.legacy.allowCreatingManagedTableUsingNonmptyLocation", "true") \
                .enableHiveSupport() \
                .master("local[*]") \
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/11 16:30:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/11 16:30:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Load config:

In [43]:
# ==========================================
# Load config
# ==========================================

#config = spark.table("db.xs0_config")

config = spark.read.parquet('xs0_config.parquet')

time_series_prep_sdate = config.collect()[0][0]
time_series_prep_sdate = config.collect()[0][0]
time_series_prep_edate = config.collect()[0][1]
train_start_date = config.collect()[0][2]
train_end_date = config.collect()[0][3]
test_start_date = config.collect()[0][4]
test_end_date = config.collect()[0][5]
score_date = config.collect()[0][6]
model_version_prefix = config.collect()[0][7]

config.show()

+----------------------+----------------------+----------------+--------------+---------------+-------------+----------+--------------------+
|time_series_prep_sdate|time_series_prep_edate|train_start_date|train_end_date|test_start_date|test_end_date|score_date|model_version_prefix|
+----------------------+----------------------+----------------+--------------+---------------+-------------+----------+--------------------+
|            2021-04-01|            2023-09-14|      2022-06-01|    2023-06-01|     2023-07-01|   2023-09-01|2023-10-01|            20230717|
+----------------------+----------------------+----------------+--------------+---------------+-------------+----------+--------------------+



## Account Transactions

In [51]:
import random
import pandas as pd
from datetime import datetime, timedelta

# Define the number of observations
num_observations = 1000000

# Generate random dates within a specific range
def random_date(start_date, end_date):
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + timedelta(days=random_number_of_days)
    return random_date

# Generate random customer numbers and transaction amounts
customer_nos = [random.randint(1000, 9999) for _ in range(num_observations)]
trn_dates = [random_date(datetime(2020, 1, 1), datetime(2023, 1, 1)) for _ in range(num_observations)]
lcy_amounts = [random.uniform(0, 1000) for _ in range(num_observations)]
drcr_inds = ['D' if random.random() < 0.5 else 'C' for _ in range(num_observations)]
ac_branches = [random.randint(1, 10) for _ in range(num_observations)]
financial_cycles = [random.randint(1, 12) for _ in range(num_observations)]
period_codes = [random.randint(1, 5) for _ in range(num_observations)]

# Create a DataFrame
data = {
    "trn_dt": trn_dates,
    "lcy_amount": lcy_amounts,
    "drcr_ind": drcr_inds,
    "ac_branch": ac_branches,
    "financial_cycle": financial_cycles,
    "period_code": period_codes
}

df = pd.DataFrame(data)
df["customer_no"] = customer_nos
df["sign_D"] = (df["drcr_ind"] == "D").astype(int)
df["sign_C"] = 1 - df["sign_D"]
df["lcy_dr"] = df["lcy_amount"] * df["sign_D"]
df["lcy_cr"] = df["lcy_amount"] * df["sign_C"]

# Extract year and month from trn_dt
df["year"] = df["trn_dt"].dt.year
df["month"] = df["trn_dt"].dt.month

# Save the synthetic dataset as a Parquet file
df.to_parquet("df_ubs_sttm_cust_account.parquet")

In [64]:
from dateutil.parser import parse

#==================
#
# 0. Columns for selection, years, months
#
#=================

columns_for_selection = ["cust_no", "trn_dt", "lcy_amount", "sign_D", "ac_branch", "financial_cycle", "period_code"]
# years = ['2018', '2019', '2020', '2021', '2022']
start_year = 2021
end_year = 2023
start_month = 1
end_month = 12

# months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

#==================
#
# 1. Read the table and apply filters
#
#=================

table = spark.read.parquet("df_ubs_sttm_cust_account.parquet").select(
    "trn_dt",
    "lcy_amount",
    "drcr_ind",
    "customer_no",
    F.col("year").alias("trn_year"),
    F.col("month").alias("trn_month")
).filter((F.col('trn_year') >= start_year) & (F.col('trn_year') <= end_year) & (F.col('trn_month') >= start_month) & (F.col('trn_month') <= end_month))

table = table\
        .withColumn('sign_D', F.when(col('drcr_ind')=='D', 1).otherwise(0))\
        .withColumn('sign_C', (1-F.col("sign_D")))\
        .withColumn("lcy_dr", F.col("lcy_amount") * F.col("sign_D"))\
        .withColumn("lcy_cr", F.col("lcy_amount") * F.col("sign_C"))

#==================
#
# 2. Apply groupBy operation to get customer year-month
#
#=================

cols_for_final_table = ["lcy_dr","lcy_cr","sign_D","sign_C","customer_no","trn_year","trn_month"]

table_grouped = table.select(cols_for_final_table).groupBy(["customer_no", "trn_year", "trn_month"]).agg(
    F.sum("lcy_dr").alias("trn_sum_D"),
    F.sum("lcy_cr").alias("trn_sum_c"),
    F.sum("sign_D").alias("trn_count_D"),
    F.sum("sign_C").alias("trn_count_C")
).write.mode('overwrite').parquet("xs2_account_transactions_grouped_"+model_version_prefix+".parquet")


# Step 2 - create complete tseries

# Convert string dates to date objects
sdate = parse(time_series_prep_sdate).date()
edate = parse(time_series_prep_edate).date()

dates = []
while sdate <= edate:
    dates.append(sdate)
    sdate = sdate + relativedelta(months=1)

dates_df = spark.createDataFrame([(d,) for d in dates], ['__PERIOD'])

df_transactions = spark.read.parquet("xs2_account_transactions_grouped_"+model_version_prefix+".parquet")

# # NEED TO CREATE A __PERIOD FIELD FROM trn_year and trn_month in the format yyyy-mm-dd where dd is always 01
df_transactions = df_transactions.withColumn("__PERIOD", F.to_timestamp(F.concat(F.col("trn_year"), F.lit("-"), F.col("trn_month"), F.lit("-01"))))
df_tseries = df_transactions.select("customer_no").distinct().crossJoin(dates_df).orderBy(["customer_no","__PERIOD"])
df_tseries = df_tseries.join(df_transactions, ["customer_no","__PERIOD"], "left").fillna(0)
df_tseries.write.mode('overwrite').parquet("xs2_account_transactions_tseries_"+model_version_prefix+".parquet")

# Step 3 - calculate moving averages

# 6) Calculate MAs
df_tseries = spark.read.parquet("xs2_account_transactions_tseries_"+model_version_prefix+".parquet")

# Get MAs for each column
value_cols = [
    "DR_total",
    "DR_min",
    "DR_max",
    "DR_count",
    "CR_total",
    "CR_min",
    "CR_max",
    "CR_count",
    "TR_count"
]

      
            
w1 = Window()\
    .partitionBy(F.col("customer_no"))\
    .orderBy(F.col("__PERIOD"))\
    .rowsBetween(-2, 0)   
    
w2 = Window()\
    .partitionBy(F.col("customer_no"))\
    .orderBy(F.col("__PERIOD"))\
    .rowsBetween(-5, -3)
            
w3 = Window()\
    .partitionBy(F.col("customer_no"))\
    .orderBy(F.col("__PERIOD"))\
    .rowsBetween(-11, -6)
            
final_table = df_tseries.withColumn("tfeat_DR_total_1_to_3m",  F.avg("trn_sum_D").over(w1))\
                       .withColumn("tfeat_DR_total_7_to_12m",  F.avg("trn_sum_D").over(w3))\
                       .withColumn("tfeat_DR_SPIKE",  F.col("trn_sum_D")/F.col("tfeat_DR_total_1_to_3m"))\
                       .withColumn("tfeat_DR_TREND",  F.col("tfeat_DR_total_1_to_3m")/F.col("tfeat_DR_total_7_to_12m"))\
                       .withColumn("tfeat_CR_total_1_to_3m",  F.avg("trn_sum_c").over(w1))\
                       .withColumn("tfeat_CR_total_7_to_12m",  F.avg("trn_sum_c").over(w3))\
                       .withColumn("tfeat_CR_SPIKE",  F.col("trn_sum_c")/F.col("tfeat_CR_total_1_to_3m"))\
                       .withColumn("tfeat_CR_TREND",  F.col("tfeat_CR_total_1_to_3m")/F.col("tfeat_CR_total_7_to_12m"))\
                       .withColumn("tfeat_DRCount_average_1_to_3m", F.avg("trn_count_D").over(w1))\
                       .withColumn("tfeat_DRCount_average_7_to_12m", F.avg("trn_count_D").over(w3))\
                       .withColumn("tfeat_CRCount_average_1_to_3m", F.avg("trn_count_C").over(w1))\
                       .withColumn("tfeat_CRCount_average_7_to_12m", F.avg("trn_count_C").over(w3))


final_table = final_table.withColumn("__PERIOD", F.add_months("__PERIOD",1))

# save the final table
final_table.write.mode('overwrite').parquet("xs2_account_transactions_final_"+model_version_prefix+".parquet")


23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/11 18:00:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

##  Credit cards transactions

##  Customer and Products

## Bank Online Usage

## Loan History

## Customer 360